# Reading data from csv via queue and threading

In [1]:
!wc -l iris_training.csv
!head -5 iris_training.csv

     121 iris_training.csv
120,4,setosa,versicolor,virginica
6.4,2.8,5.6,2.2,2
5.0,2.3,3.3,1.0,1
4.9,2.5,4.5,1.7,2
4.9,3.1,1.5,0.1,0


## reading a single record

In [1]:
import tensorflow as tf

filename_queue = tf.train.string_input_producer(['iris_training.csv'], num_epochs=2)

reader = tf.TextLineReader(skip_header_lines=1)
key, val = reader.read(filename_queue)

record_defaults = [[0.], [0.], [0.], [0.], [0.]]
cols = (col1_sepal_length, 
        col2_sepal_width, 
        col3_petal_length, 
        col4_petal_width,
        col5_species) = tf.decode_csv(val, record_defaults=record_defaults)
features = tf.stack(cols[:-1])
print 'features', features

example_batch, label_batch = tf.train.shuffle_batch(
    [features, col5_species],
    batch_size=3,
    capacity=100,
    min_after_dequeue=50)
print 'example_batch', example_batch, 'label_batch', label_batch

features Tensor("stack:0", shape=(4,), dtype=float32)
example_batch Tensor("shuffle_batch:0", shape=(3, 4), dtype=float32) label_batch Tensor("shuffle_batch:1", shape=(3,), dtype=float32)


decode_csv的操作是对一行进行的
前4列组成了features，最后一列是分类标签

In [2]:
s = tf.Session()
s.run(tf.local_variables_initializer())

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=s, coord=coord)
print 'num of threads', len(threads)

num of threads 4


- 有多少threads不知道是怎么控制的?
- string_input_producer设置了num_epochs以后，需要调用local_variables_initializer，不明白

In [3]:
for i in xrange(2):
    example, label = s.run([features, col5_species])
    print example, label

for i in xrange(2):
    print s.run([example_batch, label_batch])

[ 4.9000001   2.4000001   3.29999995  1.        ] 1.0
[ 4.5         2.29999995  1.29999995  0.30000001] 0.0
[array([[ 4.4000001 ,  3.        ,  1.29999995,  0.2       ],
       [ 6.30000019,  2.5       ,  5.        ,  1.89999998],
       [ 4.9000001 ,  3.0999999 ,  1.5       ,  0.1       ],
       [ 6.30000019,  2.29999995,  4.4000001 ,  1.29999995],
       [ 4.69999981,  3.20000005,  1.60000002,  0.2       ]], dtype=float32), array([ 0.,  2.,  0.,  1.,  0.], dtype=float32)]
[array([[ 4.9000001 ,  3.0999999 ,  1.5       ,  0.1       ],
       [ 6.69999981,  3.        ,  5.        ,  1.70000005],
       [ 5.0999999 ,  3.79999995,  1.5       ,  0.30000001],
       [ 4.69999981,  3.20000005,  1.29999995,  0.2       ],
       [ 5.9000001 ,  3.        ,  5.0999999 ,  1.79999995]], dtype=float32), array([ 0.,  1.,  0.,  0.,  2.], dtype=float32)]


因为decode_csv是对行进行操作的，所以每次run就返回文件中的一行
而example_batch每次返回一批数据，还会shuffle

In [3]:
import sys
n = 0
try:
    while not coord.should_stop():
        X, y_ = s.run([example_batch, label_batch])
        n += X.shape[0]
        sys.stdout.write('.')
        sys.stdout.flush()
except tf.errors.OutOfRangeError:
    print  # reach epoch limits
print 'total', n

................................................................................
total 240


## closing

In [4]:
coord.request_stop()
coord.join(threads)